## INF2209 Midterm Submission

#### Hong Shi         1002685070

In [1]:
# SETUP
import numpy as np
import pandas as pd
import os
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [2]:
import gensim
import gensim.corpora as corpora
from pprint import pprint

In [3]:
# Read in the raw data
raw_ti10 = pd.read_csv('new_ti10_tweets.csv', encoding = 'utf-8')

D:\Efficiency_softwares\Anaconda\AnaInstall\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


#### Breifly examine the data

In [4]:
pd.set_option("display.max_columns", None)
raw_ti10.head()

user_id             status_id           created_at  \
0  x1290210901543596035  x1450265609837625353  2021-10-19 01:00:40   
1           x1227015854  x1450265585749741569  2021-10-19 01:00:34   
2           x1227015854  x1449897704268697604  2021-10-18 00:38:45   
3           x1227015854  x1449756111268163591  2021-10-17 15:16:06   
4           x1227015854  x1449871356200464394  2021-10-17 22:54:03   

     screen_name                                               text  \
0  ANIMATIONWEB5  #alterpinay #LISA #Attacca #TI10 #honktwtselfi...   
1    Dota2Lounge  Miposhka shared in the team's vlog what really...   
2    Dota2Lounge  Crazy to think that #TI10 is over. These were ...   
3    Dota2Lounge  The level of performance by both teams is amaz...   
4    Dota2Lounge  This meta <U+0001F923> \r\n\r\n#dota2 #ti10 ht...   

                source  display_text_width reply_to_status_id  \
0  Twitter for Android                 279                NaN   
1               Buffer                 118                NaN   
2      Twitter Web App                 107                NaN   
3               Buffer                  64                NaN   
4               Buffer                  26                NaN   

  reply_to_user_id reply_to_screen_name  is_quote  is_retweet  favorite_count  \
0              NaN                  NaN     False       False               1   
1              NaN                  NaN     False       False               2   
2              NaN                  NaN     False       False               5   
3              NaN                  NaN     False       False               0   
4              NaN                  NaN     False       False               8   

   retweet_count  quote_count  reply_count  \
0              1          NaN          NaN   
1              0          NaN          NaN   
2              2          NaN          NaN   
3              1          NaN          NaN   
4              1          NaN          NaN   

                                            hashtags symbols urls_url  \
0  alterpinay LISA Attacca TI10 honktwtselfieday ...     NaN      NaN   
1                                         TI10 dota2     NaN      NaN   
2                                               TI10     NaN      NaN   
3                                         dota2 ti10     NaN      NaN   
4                                         dota2 ti10     NaN      NaN   

  urls_t.co urls_expanded_url  \
0       NaN               NaN   
1       NaN               NaN   
2       NaN               NaN   
3       NaN               NaN   
4       NaN               NaN   

                                           media_url               media_t.co  \
0     http://pbs.twimg.com/media/FCBg4sLXsAgUmQi.jpg  https://t.co/MS6GpBxMD6   
1  http://pbs.twimg.com/ext_tw_video_thumb/145026...  https://t.co/oIzjjGuNDW   
2     http://pbs.twimg.com/media/FB8Rd2CX0AELMGp.jpg  https://t.co/e9gQ15Qgo6   
3  http://pbs.twimg.com/ext_tw_video_thumb/144975...  https://t.co/TUOjs0xXe9   
4     http://pbs.twimg.com/media/FB76UIUXEAEqOqn.jpg  https://t.co/d0GEdB9wI1   

                                  media_expanded_url media_type  \
0  https://twitter.com/ANIMATIONWEB5/status/14502...      photo   
1  https://twitter.com/Dota2Lounge/status/1450265...      photo   
2  https://twitter.com/Dota2Lounge/status/1449897...      photo   
3  https://twitter.com/Dota2Lounge/status/1449756...      photo   
4  https://twitter.com/Dota2Lounge/status/1449871...      photo   

                                       ext_media_url           ext_media_t.co  \
0     http://pbs.twimg.com/media/FCBg4sLXsAgUmQi.jpg  https://t.co/MS6GpBxMD6   
1  http://pbs.twimg.com/ext_tw_video_thumb/145026...  https://t.co/oIzjjGuNDW   
2     http://pbs.twimg.com/media/FB8Rd2CX0AELMGp.jpg  https://t.co/e9gQ15Qgo6   
3  http://pbs.twimg.com/ext_tw_video_thumb/144975...  https://t.co/TUOjs0xXe9   
4     http://pbs.twimg.com/media/FB76UIUXEAEqOqn.jpg  https://t.co/d0GEdB9wI1   

In [5]:
# Size of the raw dataset
raw_ti10.shape

(41839, 90)

#### Preprocessing

##### 1. Keep only English tweets

In [6]:
raw_ti10.lang.unique()

array(['und', 'en', 'es', 'pt', 'in', 'ru', 'zh', 'tl', 'sl', 'th', 'ro',
       'ar', 'nl', 'fa', 'de', 'fr', 'cs', 'ja', 'lv', 'sv', 'et', 'da',
       'fi', 'eu', 'bg', 'it', 'no', 'hi', 'is', 'uk', 'vi', 'ca', 'ht',
       'pl', 'cy', 'hu', 'lt', 'ko', 'sr', 'tr', 'si'], dtype=object)

In [7]:
en_ti10 = raw_ti10[raw_ti10["lang"] == 'en']
# There are 26839 English tweets defined by Twitter API
en_ti10.shape[0]

26839

##### 2. Remove URLs

In [8]:
#sample tweet raw text in text column of the data
test_text = "@Team__Spirit Miposhka shared in the team's vlog what really helped the team to achieve their well-deserved victory at #TI10\r\n\r\n#dota2 https://t.co/oIzjjGuNDW"

In [9]:
def remove_URL(text):
    """Remove URLs from tweet text"""
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"www.\S+", "", text)
    return text

In [19]:
# function test
test_text = remove_URL(test_text)
test_text

"@Team__Spirit Miposhka shared in the team's vlog what really helped the team to achieve their well-deserved victory at #TI10\r\n\r\n#dota2 "

##### 3. Remove Hashtags

In [11]:
# Example of tweets with hashtags (add a "#" symbol before a phrase, e.g. #Ti10)
en_ti10[en_ti10.text.str.contains('#')].text.iloc[0]

"Miposhka shared in the team's vlog what really helped the team to achieve their well-deserved victory at #TI10\r\n\r\n#dota2 https://t.co/oIzjjGuNDW"

In [12]:
# Example of tweets with mentions (add a "@" symbol before a tweet user, e.g. @EvilGeniuses)
en_ti10[en_ti10.text.str.contains('@')].text.iloc[1]

"The only one I managed to win, thanks to bulba's drafts.\r\n@EvilGeniuses #Dota2 #TI10 https://t.co/W4qlXkKzBE"

In [14]:
# hashtags always contain the search term #ti10, and the top frequenct hashtags contain DOTA2 game name, which is already within the data collection context
en_ti10.hashtags.value_counts()

TI10                                                                                                               17356
TI10 Dota2                                                                                                          1341
ti10                                                                                                                 965
Dota2 TI10                                                                                                           482
DreamOG TI10                                                                                                         300
                                                                                                                   ...  
Revolotto shibainucoin alts shibarium token AirdropKing SolanaNFTs Litecoin TI10 GFvip Bitcoin BlockchainGaming        1
TI10 dota2 esports                                                                                                     1
T1WIN TI10 T1Fighting igniT1on  

In [15]:
# mentions mostly contatins different teams participating TI10, e.g.Team_Spirit
en_ti10.mentions_screen_name.value_counts()

Team__Spirit                                              1598
OGesports                                                  431
teamsecret                                                 374
T1                                                         323
EvilGeniuses                                               126
                                                          ... 
dota2ti jonas_volek                                          1
Team__Spirit TeamLiquid NigmaGalaxy LGDgaming newbeecn       1
drinkrelaxplay                                               1
Team__Spirit PSGeSports dota2ti DOTA2 JenkinsDota            1
LGDgaming teamsecret Team__Spirit                            1
Name: mentions_screen_name, Length: 1459, dtype: int64

Would like to remove only hashtags but not the mentions

In [16]:
def remove_hashtag(text):
    """Remove hashtags from tweet text"""
    return re.sub(r"#\S+", "", text)

In [20]:
# function test
test_text = remove_hashtag(test_text)
test_text

"@Team__Spirit Miposhka shared in the team's vlog what really helped the team to achieve their well-deserved victory at \r\n\r\n "

##### 4. Remove Emojis (Unicode characters)

In [17]:
# example of tweets with unicode characters
# <U+0001F923> belongs to an emoji symbol
en_ti10.text.iloc[3]

'This meta <U+0001F923> \r\n\r\n#dota2 #ti10 https://t.co/d0GEdB9wI1'

In [18]:
def remove_unicode_char(text):
    """Remove unicode character from tweet text"""
    return re.sub(r"<U\+\S+", "", text)

In [21]:
# function test
remove_unicode_char(en_ti10.text.iloc[3])

'This meta  \r\n\r\n#dota2 #ti10 https://t.co/d0GEdB9wI1'

##### 5. Remove newlines

In [22]:
def remove_newline_char(text):
    """Remove newline character from tweet text"""
    text = re.sub(r"\r", "", text)
    text = re.sub(r"\n", "", text)
    return text

In [23]:
# function test
test_text = remove_newline_char(test_text)
test_text

"@Team__Spirit Miposhka shared in the team's vlog what really helped the team to achieve their well-deserved victory at  "

##### 6. Remove punctuations and lowercase characters

In [24]:
def no_punc_lower(text):
    """Remove punctuations and lowercase characters from tweet text"""
    text = re.sub("'","", text) # to avoid removing contraction
    text = re.sub("\W", " ", text).lower()
    return text

In [25]:
# function test
test_text = no_punc_lower(test_text)
test_text

' team__spirit miposhka shared in the teams vlog what really helped the team to achieve their well deserved victory at  '

##### 7. Remove Stopwords

In [26]:
stop_words = set(stopwords.words('english'))

In [31]:
 """Remove NLTK stopwords from tweet text"""
def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in (stop_words)])

In [32]:
# function test
test_text = remove_stopwords(test_text)
test_text

'team__spirit miposhka shared teams vlog really helped team achieve well deserved victory'

##### 8. Tokenize the sentence and get the lemma of tokens

Tokenization could be achieved through word_tokenize function in nltk

In [33]:
def get_pos_tags(words):
    '''Get the part of speech for lemmatization''' 
    tags=[]
    for word in words:
        tags.append(nltk.pos_tag([word]))
    return tags

In [36]:
def get_lemma(word_tags):
    '''Lemmatize the word'''
    wordnet_lemmatizer = WordNetLemmatizer()
    lemma = []
    for element in word_tags:
        word = element[0][0]
        pos = element[0][1]
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ,
                   "N": wordnet.NOUN,
                   "V": wordnet.VERB,
                   "R": wordnet.ADV}
        wordnet_pos = tag_dict.get(tag, wordnet.NOUN)
        lemma.append(wordnet_lemmatizer.lemmatize(word, wordnet_pos))
    return(lemma)

In [37]:
# function_test
get_lemma(get_pos_tags(word_tokenize(test_text)))

['team__spirit',
 'miposhka',
 'share',
 'team',
 'vlog',
 'really',
 'help',
 'team',
 'achieve',
 'well',
 'deserve',
 'victory']

##### Apply all of the above functions on the raw tweet text in sequence to clean data

In [38]:
en_ti10["clean_text"] = \
en_ti10["text"].apply(remove_URL).apply(remove_hashtag).apply(remove_unicode_char).apply(remove_newline_char).apply(no_punc_lower).apply(remove_stopwords)

<ipython-input-38-429975f85f56>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_ti10["clean_text"] = \


In [42]:
# sample output
en_ti10.clean_text.iloc[0:4]

1    miposhka shared teams vlog really helped team ...
2         crazy think picked heroes think needs tuning
3                      level performance teams amazing
4                                                 meta
Name: clean_text, dtype: object

In [40]:
en_ti10["clean_text_token"] = \
en_ti10["clean_text"].apply(word_tokenize).apply(get_pos_tags).apply(get_lemma)

<ipython-input-40-b487a434a66a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_ti10["clean_text_token"] = \


In [43]:
# sample output
en_ti10.clean_text_token.iloc[0:4]

1    [miposhka, share, team, vlog, really, help, te...
2      [crazy, think, picked, hero, think, need, tune]
3                    [level, performance, team, amaze]
4                                               [meta]
Name: clean_text_token, dtype: object

#### Draft on Topic Model 

In [44]:
# would like to try LDA first as BTM model is still not introduced in the lecture
id2word = corpora.Dictionary(en_ti10.clean_text_token)
texts = en_ti10.clean_text_token
corpus = [id2word.doc2bow(text) for text in texts]

In [47]:
num_topics = 5

lda_model = gensim.models.LdaMulticore(corpus=corpus,id2word=id2word,num_topics=num_topics)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.024*"game" + 0.017*"team" + 0.014*"ogesports" + 0.013*"go" + 0.011*"let" '
  '+ 0.010*"watch" + 0.010*"win" + 0.009*"good" + 0.009*"2" + 0.009*"og"'),
 (1,
  '0.026*"game" + 0.022*"team__spirit" + 0.014*"team" + 0.011*"win" + '
  '0.010*"2" + 0.010*"bracket" + 0.009*"take" + 0.009*"spirit" + 0.008*"dota" '
  '+ 0.008*"go"'),
 (2,
  '0.047*"team" + 0.034*"spirit" + 0.029*"game" + 0.016*"v" + 0.013*"ti" + '
  '0.009*"og" + 0.008*"team__spirit" + 0.008*"dota" + 0.008*"secret" + '
  '0.007*"lgd"'),
 (3,
  '0.023*"team" + 0.020*"go" + 0.016*"spirit" + 0.014*"let" + 0.012*"get" + '
  '0.009*"bracket" + 0.009*"t1" + 0.008*"lgd" + 0.008*"og" + 0.007*"like"'),
 (4,
  '0.025*"game" + 0.019*"final" + 0.017*"go" + 0.016*"t1" + 0.011*"let" + '
  '0.011*"2" + 0.010*"grand" + 0.009*"v" + 0.009*"team__spirit" + '
  '0.008*"team"')]
